In [242]:
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df = pd.read_csv('the-reddit-nft-dataset-comments.csv')

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8285561 entries, 0 to 8285560
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   type            object 
 1   id              object 
 2   subreddit.id    object 
 3   subreddit.name  object 
 4   subreddit.nsfw  object 
 5   created_utc     float64
 6   permalink       object 
 7   body            object 
 8   sentiment       float64
 9   score           float64
dtypes: float64(3), object(7)
memory usage: 632.1+ MB


In [212]:
testing = df.sample(50).dropna().drop(columns=['id','subreddit.id','created_utc','permalink','sentiment','score'])

In [213]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punctuation = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
random = lambda x: re.sub('\n',' ',x)
lower_case = lambda x: x.lower()

In [214]:
testing['body'] = testing.body.map(alphanumeric).map(punctuation).map(lower_case).map(random)

In [215]:
testing

,type,subreddit.name,subreddit.nsfw,body
167934,comment,pcgaming,False,its impossible have nfts in a video game that ...
8221092,comment,cryptocurrency,False,if this sells i m done with nfts haha cmon dude
7726309,comment,cryptocurrency,False,i guess all the nft porn platforms are celebra...
6716877,comment,superstonk,False,i do get where you are coming from but rc is ...
4078370,comment,nftsmarketplace,False,my metamask key gt i need this nft
333147,comment,pathfinder2e,False,here s a sneak peek of r dndnext using the t...
4081622,comment,nftsmarketplace,False,my metamask key gt i want won this nft
7024798,comment,nft,False,thank you for your submission on r nft join u...
5952106,comment,crypto_com,False,if he means the bad type of nfts you re right ...
4076136,comment,shitposting,False,dude i own this nft do you really think that ...


In [217]:
comments = testing.body

In [218]:
vec = CountVectorizer()

In [219]:
comments_term = vec.fit_transform(comments)

comments_term.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 3, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [220]:
pd.unique(comments_term.toarray().reshape(-1))

array([0, 1, 2, 5, 3, 4, 7, 8, 6])

In [230]:
vec.vocabulary_

{'its': 135,
 'impossible': 126,
 'have': 108,
 'nfts': 174,
 'in': 127,
 'video': 279,
 'game': 91,
 'that': 252,
 'are': 11,
 'healthy': 110,
 'for': 86,
 'the': 253,
 'and': 9,
 'consumer': 52,
 'period': 194,
 'just': 139,
 'fucking': 89,
 'give': 100,
 'up': 273,
 'already': 5,
 'if': 123,
 'this': 262,
 'sells': 229,
 'done': 66,
 'with': 290,
 'haha': 106,
 'cmon': 41,
 'dude': 68,
 'guess': 105,
 'all': 4,
 'nft': 173,
 'porn': 201,
 'platforms': 197,
 'celebrating': 40,
 'today': 266,
 'do': 65,
 'get': 96,
 'where': 287,
 'you': 297,
 'coming': 45,
 'from': 88,
 'but': 33,
 'rc': 210,
 'is': 133,
 'an': 8,
 'entrepreneur': 71,
 'by': 35,
 'heart': 111,
 'he': 109,
 'were': 284,
 'someone': 239,
 'who': 288,
 'was': 281,
 'huge': 121,
 'crypto': 56,
 'then': 257,
 'your': 298,
 'train': 268,
 'of': 178,
 'thought': 263,
 'may': 160,
 'been': 22,
 'highly': 113,
 'possible': 202,
 'would': 294,
 'loopring': 149,
 'at': 15,
 'start': 242,
 'finding': 83,
 'way': 283,
 'to': 265,

In [223]:
comments_term.shape

(13, 299)

In [224]:
stop_words = CountVectorizer(stop_words='english',
                      max_df=0.8)

In [225]:
comments_term = stop_words.fit_transform(comments)

comments_term.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [233]:
comments_term.shape

(13, 209)

In [240]:
pd.DataFrame(comments_term.toarray(), index=comments, columns=stop_words.get_feature_names_out())

,action,actually,advise,amp,ask,ass,automatically,aware,away,bad,...,validate,ve,video,want,wasn,way,won,world,www,year
body,,,,,,,,,,,,,,,,,,,,,
its impossible have nfts in a video game that are healthy for the game and the consumer period just fucking give up already,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
if this sells i m done with nfts haha cmon dude,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i guess all the nft porn platforms are celebrating today,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
i do get where you are coming from but rc is an entrepreneur by heart if he were someone who was huge in crypto then your train of thought may have been highly possible he would have been with loopring at the start and is finding a way to place loopring on the world map by making use of its for low gas fees to transact nfts however as far as i am aware there wasn t much of a connection between rc and any form of crypto amp i do still feel that rc himself bought into gamestop because of the potential growth in the gaming industry as per his letter to the ex gamestop board gamestop is still a household name in the us and many parts of europe he sees the potential of nfts within the gaming industry and is partnering with loopring to do just that when this takes off both loopring and gamestop stands to gain,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,1,1,0,1,0,0
my metamask key gt i need this nft,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
here s a sneak peek of r dndnext using the top posts https np reddit com r dndnext top sort top amp t year of the year no nfts https np reddit com r dndnext comments no nfts a player stole important info about my campaign what do i do https np reddit com r dndnext comments a player stole important info about my campaign when it comes to rules what makes you say i recognize that the council has made a decision but given that it s a stupid ass decision i ve elected to ignore it https np reddit com r dndnext comments when it comes to rules what makes you say i i m a bot beep boop downvote to remove contact https www reddit com message compose to sneakpeekbot info https np reddit com r sneakpeekbot opt out https np reddit com r sneakpeekbot comments blacklist ix github https github com ghnr sneakpeekbot,0,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,2
my metamask key gt i want won this nft,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
thank you for your submission on r nft join us on discord https discord gg for live discussion on everything nfts and to share amp buy sell your nfts join r nft discord https discord gg members follow rnftcommunity twitter https twitter com rnftcommunity to get featured i am a bot and this action was performed automatically please contact the moderators of this subreddit message compose to r nft if you have any questions or concerns,1,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
if he means the bad type of nfts you re right actually no one deserves them,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [259]:
lsa = TruncatedSVD(13)
doc_topic = lsa.fit_transform(comments_term)
lsa.explained_variance_ratio_

array([0.53374584, 0.14992964, 0.11137042, 0.04959929, 0.04714325,
       0.04325151, 0.01814368, 0.01308103, 0.01089423, 0.00893508,
       0.00751604, 0.00448649, 0.00190352])

In [260]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5","component_6",
                      "component_7","component_8","component_9","component_10","component_11","component_12",
                     "component_13"],
             columns = stop_words.get_feature_names_out())
topic_word

,action,actually,advise,amp,ask,ass,automatically,aware,away,bad,...,validate,ve,video,want,wasn,way,won,world,www,year
component_1,0.009,0.001,0.001,0.061,0.001,0.052,0.009,0.001,0.002,0.000,...,0.001,0.053,0.000,0.000,0.001,0.001,0.000,0.001,0.052,0.104
component_2,0.019,0.007,0.004,0.107,0.009,-0.005,0.019,0.097,0.004,0.002,...,0.004,-0.000,0.004,0.003,0.097,0.097,0.001,0.097,-0.005,-0.009
component_3,0.141,0.006,0.003,0.070,0.007,-0.016,0.141,-0.019,0.035,0.002,...,0.003,-0.013,0.002,0.008,-0.019,-0.019,0.006,-0.019,-0.016,-0.033
component_4,0.104,0.024,0.027,-0.053,0.053,0.003,0.104,0.001,0.160,-0.002,...,0.027,0.029,-0.004,-0.001,0.001,0.001,0.001,0.001,0.003,0.006
component_5,-0.023,0.194,0.178,-0.009,0.356,-0.001,-0.023,-0.008,-0.023,0.016,...,0.178,0.177,0.010,0.006,-0.008,-0.008,-0.001,-0.008,-0.001,-0.002
component_6,-0.049,0.016,0.015,0.001,0.030,0.000,-0.049,-0.000,-0.049,0.002,...,0.015,0.015,0.001,0.009,-0.000,-0.000,0.008,-0.000,0.000,0.001
component_7,-0.004,0.028,-0.022,-0.022,-0.044,-0.001,-0.004,-0.011,0.007,0.050,...,-0.022,-0.023,0.276,0.028,-0.011,-0.011,-0.005,-0.011,-0.001,-0.002
component_8,-0.030,-0.009,-0.004,-0.020,-0.007,0.003,-0.030,0.001,-0.006,-0.005,...,-0.004,-0.001,0.010,0.260,0.001,0.001,0.231,0.001,0.003,0.006
component_9,-0.008,0.301,-0.030,-0.018,-0.060,-0.001,-0.008,-0.007,0.003,0.331,...,-0.030,-0.031,-0.092,0.094,-0.007,-0.007,0.016,-0.007,-0.001,-0.002
component_10,-0.012,0.043,-0.019,-0.008,-0.038,0.001,-0.012,0.001,-0.002,0.062,...,-0.019,-0.018,0.003,-0.132,0.001,0.001,-0.098,0.001,0.001,0.003


In [253]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [254]:
display_topics(lsa, stop_words.get_feature_names_out(), 5)


Topic  0
https, com, reddit, np, dndnext

Topic  1
gamestop, loopring, rc, nfts, crypto

Topic  2
discord, nft, twitter, rnftcommunity, gg

Topic  3
really, case, stole, say, dude

Topic  4
ask, nfts, actually, guess, advise

Topic  5
icicb, using, recently, games, know

Topic  6
game, nfts, consumer, period, healthy

Topic  7
gt, metamask, key, nft, want

Topic  8
nfts, deserves, means, right, bad

Topic  9
celebrating, today, porn, platforms, guess
